In [ ]:
import numpy as np
import pandas as pd
traind=pd.read_csv(r'carstrain_2.csv')
train=traind.drop(columns='Unnamed: 0')
testd=pd.read_csv(r'carstest_2.csv')
test=testd.drop(columns='Unnamed: 0')
train=train.dropna(axis=0)
train.index=range(train.shape[0])
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(categories='auto').fit(train.loc[:,'State'].values.reshape(-1,1))
result=OneHotEncoder(categories='auto').fit_transform(train.loc[:,'State'].values.reshape(-1,1))
result1=result.toarray()
newtrain=pd.concat([train,pd.DataFrame(result1)],axis=1)
newtrain.columns=['is_severe', 'State', 'std.boxcox.distance', 'Side', 'Day',
       'std.temp', 'std.boxcox.humidity', 'std.boxcox.pressure',
       'std.boxcox.windspeed', 'std.boxcox.precipitation', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming','Traffic_Signal', 'Turning_Loop','x0_AZ', 'x0_CA', 'x0_FL', 'x0_MN', 'x0_NC', 'x0_NY', 'x0_OR',
       'x0_SC', 'x0_TX', 'x0_VA']
resulttest=OneHotEncoder(categories='auto').fit_transform(test.loc[:,'State'].values.reshape(-1,1))

resultt=resulttest.toarray()
newtest=pd.concat([test,pd.DataFrame(resultt)],axis=1)
newtest.columns=['is_severe', 'State', 'std.boxcox.distance', 'Side', 'Day',
       'std.temp', 'std.boxcox.humidity', 'std.boxcox.pressure',
       'std.boxcox.windspeed', 'std.boxcox.precipitation', 'Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming','Traffic_Signal', 'Turning_Loop','x0_AZ', 'x0_CA', 'x0_FL', 'x0_MN', 'x0_NC', 'x0_NY', 'x0_OR',
       'x0_SC', 'x0_TX', 'x0_VA']
from sklearn.preprocessing import OrdinalEncoder
newtrain.iloc[:,0]=OrdinalEncoder().fit_transform(newtrain.iloc[:,0].values.reshape(-1,1))
newtest.iloc[:,0]=OrdinalEncoder().fit_transform(newtest.iloc[:,0].values.reshape(-1,1))
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier as RFC
y1=newtrain.iloc[:,0].values
newtrain=newtrain.drop(columns='is_severe')
y2=newtest.iloc[:,0].values
newtest=newtest.drop(columns='is_severe')
newtrain.iloc[:,1]=OrdinalEncoder().fit_transform(newtrain.iloc[:,1].values.reshape(-1,1))
newtest.iloc[:,1]=OrdinalEncoder().fit_transform(newtest.iloc[:,1].values.reshape(-1,1))
newtrain=newtrain.drop(columns='State')
newtrain.Side=np.where(newtrain.Side == 'R', 1, 0)
newtest=newtest.drop(columns='State')
newtest.Side=np.where(newtest.Side == 'R', 1, 0)
RFC_=RFC(n_estimators=100,random_state=0)
x_embedded=SelectFromModel(RFC_).fit_transform(newtrain,y1)
tenbed=newtest.iloc[:,[0,1,3,4,5,6]]
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
clf=DecisionTreeClassifier(random_state=0)

clf=clf.fit(x_embedded,y1)
RFC_=RFC_.fit(x_embedded,y1)

score_c=clf.score(tenbed,y2)
score_r=RFC_.score(tenbed,y2)

print("single tree:{}".format(score_c),
     "random forest:{}".format(score_r))
clf=DecisionTreeClassifier(random_state=0)

clf=clf.fit(newtrain,y1)
RFC_=RFC_.fit(newtrain,y1)

score_c=clf.score(newtest,y2)
score_r=RFC_.score(newtest,y2)

print("single tree:{}".format(score_c),
     "random forest:{}".format(score_r))
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score
lrl2=LR(penalty="l2",solver="liblinear",C=0.5,max_iter=1000)
lrl2=lrl2.fit(newtrain,y1)
lrl2.coef_
accuracy_score(lrl2.predict(newtest),y2)
y_score = RFC_.predict_proba(newtest)
y_score_lrl2 = lrl2.predict_proba(newtest)
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
#ROC
fpr, tpr, thresholds = roc_curve(y2, y_score[:, 1])
roc_auc = auc(fpr, tpr)
def drawRoc(roc_auc,fpr,tpr):
    plt.subplots(figsize=(7, 5.5))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
drawRoc(roc_auc, fpr, tpr)
from sklearn.metrics import confusion_matrix
#Random Forest confusion matrix
conf_matrix_rf=confusion_matrix(y2, RFC_.predict(newtest))
conf_matrix_rf
#sensitivity
conf_matrix_rf[1,1]/np.sum(conf_matrix_rf[:,1])
#specificity 
conf_matrix_rf[0,0]/np.sum(conf_matrix_rf[:,0])
#Logistic Regression confusion matrix
conf_matrix_lrl2=confusion_matrix(y2, lrl2.predict(newtest))
conf_matrix_lrl2
#sensitivity
conf_matrix_lrl2[1,1]/np.sum(conf_matrix_lrl2[:,1])
#specificity 
conf_matrix_lrl2[0,0]/np.sum(conf_matrix_lrl2[:,0])

